In [2]:
### cargar las librerias necesarias 
suppressMessages(library(readxl))
suppressMessages(library(caret))

In [6]:
### cargar la tabla con la totalidad de los datos creada previamente
data = read_excel("D:/GEOBIA/data.xlsx",col_names = T)
### selecciona las columnas de interés y asegura borrar datos nulos 
data_all = na.omit(as.data.frame(data[,c(8:27)]))
### se pide que de los nombres para asegurar que sean las columnas correctas
names(data_all)

[1] "cp1_mean"   "cp1_median" "cp1_stdev"  "cp1_majori" "cp1_varian"
 [6] "cp2_mean"   "cp2_median" "cp2_stdev"  "cp2_majori" "cp2_varian"
[11] "cp3_mean"   "cp3_median" "cp3_stdev"  "cp3_majori" "cp3_varian"
[16] "cp4_mean"   "cp4_median" "cp4_stdev"  "cp4_majori" "cp4_varian"

In [7]:
### cargar la tabla de entrenamiento y validación
data1 = read_excel("D:/GEOBIA/train.xlsx",col_names = T)
names(data1)

[1] "id"         "class"      "ID_2"       "VALUE"      "NAME"      
 [6] "pci [0] (M" "pci [1] (M" "pci [2] (M" "pci [3] (M" "cp1_mean"  
[11] "cp1_median" "cp1_stdev"  "cp1_majori" "cp1_varian" "cp2_mean"  
[16] "cp2_median" "cp2_stdev"  "cp2_majori" "cp2_varian" "cp3_mean"  
[21] "cp3_median" "cp3_stdev"  "cp3_majori" "cp3_varian" "cp4_mean"  
[26] "cp4_median" "cp4_stdev"  "cp4_majori" "cp4_varian"

In [9]:
### selecciona las columnas de interés   y asegura borrar datos nulos 
data2 = na.omit(as.data.frame(data1[,c(10:29, 2)]))
names(data2)

[1] "cp1_mean"   "cp1_median" "cp1_stdev"  "cp1_majori" "cp1_varian"
 [6] "cp2_mean"   "cp2_median" "cp2_stdev"  "cp2_majori" "cp2_varian"
[11] "cp3_mean"   "cp3_median" "cp3_stdev"  "cp3_majori" "cp3_varian"
[16] "cp4_mean"   "cp4_median" "cp4_stdev"  "cp4_majori" "cp4_varian"
[21] "class"

In [11]:
### se crea una partición entrenamiento/validación con 75% de los datos para entrenamiento
set.seed(123)
trainIndex = createDataPartition(data2$class, p = .75, 
                                  list = F, 
                                  times = 1)
head(trainIndex)

Resample1
1
2
3
4
5
8


In [12]:
### separar datos de entrenamiento y de validación
dataTrain = data2[ trainIndex,]
dataTest  = data2[-trainIndex,]

In [13]:
### verificar las frecuencias de cada clase en la seleccion de entrenamiento
data.frame(table(dataTrain$class))

Var1,Freq
bosque,40
enero,60
enmalezado,51
febrero,65
humedo,52
inundado,54
matorral,63
seco,60
urbano,57


In [14]:
### verificar las frecuencias de cada clase en la seleccion de validación
data.frame(table(dataTest$class))

Var1,Freq
bosque,13
enero,19
enmalezado,17
febrero,21
humedo,17
inundado,17
matorral,21
seco,20
urbano,19


Observando las frecuencias tanto de los datos de entrenamiento como de validación, se encuentra que los datos de febrero, enero, matorral, seco, en el entrenamiento tienen mayor cantidad de datos que los datos de bosque, enmalezado y urbano, la misma tendencia se observa en los datos de validación, de acuerdo con Maxwell, Warner, & Fang, (2018), Random Forest es sensible a proporciones diferentes de clases, ya que se puede favorecer a la clase mayoritaria cuando se realiza el proceso de votación para la escogencia de la pertenencia a una clase, se presentará una mayor cantidad de votos para las clases con mayor representatividad, por tal razón se hace necesario balancear el número de muestras de las clases.

A continuación se crea una función que balancea los datos, los argumentos utilizados son los siguientes, x que hace referencia al dataset,  classCol la columna que contiene las clases y nsamples_class, que es el número de muestras.   

In [15]:
# funcion para balancear el dataset
balancea = function(x, classCol, nsamples_class){
  for (i in 1:length(unique(x[, classCol]))){
    class.i = unique(x[, classCol])[i]
    if((sum(x[, classCol] == class.i) - nsamples_class) != 0){
      x = x[-sample(which(x[, classCol] == class.i), 
                     sum(x[, classCol] == class.i) - nsamples_class), ]
    }
  }
  return(x)
}

In [27]:
### selecciona el número de observaciones para balancer los datos 40 en este caso
no_ob_class = 40
### Balancea el dataset de entrenamiento
training_balan = balancea(dataTrain, "class", no_ob_class)
### se verifica que hayan quedado con iguales datos todas las variables 
data.frame( table(training_balan$class))

Var1,Freq
bosque,40
enero,40
enmalezado,40
febrero,40
humedo,40
inundado,40
matorral,40
seco,40
urbano,40


In [28]:
# entrenamiento del Modelo Random Forests
set.seed(123)
modelo_rf = train(as.factor(class) ~ ., data = training_balan, method = "rf",importance = TRUE,verbose = TRUE)
pred.rf = predict(modelo_rf, dataTest)

In [40]:
### importancia de las variables en el modelo 
fm =modelo_rf$finalModel
c=as.matrix(round(varImp(fm),2))
c

,bosque,enero,enmalezado,febrero,humedo,inundado,matorral,seco,urbano
cp1_mean,14.78,15.28,13.80,13.69,14.23,9.98,13.29,16.72,9.89
cp1_median,14.44,15.41,13.59,13.55,13.47,10.84,12.67,17.87,9.20
cp1_stdev,5.00,7.37,8.23,10.44,6.12,5.38,9.45,2.45,10.30
cp1_majori,14.21,14.63,15.08,15.12,15.52,10.71,12.37,14.48,10.43
cp1_varian,3.82,6.64,6.92,8.20,5.25,4.10,9.04,1.54,9.42
cp2_mean,11.67,10.00,10.72,10.16,10.77,16.40,10.57,11.49,7.70
cp2_median,11.81,10.33,9.68,9.84,10.22,14.77,11.12,11.94,7.41
cp2_stdev,7.00,5.10,4.54,2.87,8.10,5.67,3.76,1.90,8.54
cp2_majori,11.20,9.14,7.35,9.27,9.25,13.59,10.18,10.89,5.94
cp2_varian,7.82,3.07,2.11,2.20,8.05,2.12,2.44,3.33,8.51


Respecto a la importancia de las variables en cada una de las clases la variable mediana del componente 1 (cp1_median) de la clase seco presentó el valor más alto (17.8), sin embargo, esta misma clase presentó el valor más bajo  (1.54) para la varianza del componente principal 1 (cp1_varian) adicionalmente en conjunto todos los fueron los más bajos de todas las clases, lo que sugiere que la mediana del componente uno es más útil para realizar la clasificación. La clase enmalezado presentó el segundo valor más alto (16.7) para la mediana del componente 4 (cp4_median), también presentó el segundo valor más bajo (2.10) para la varianza del componente 2 (cp2_var).

In [43]:
### exporta la tabla de variables de importancia 
export=as.data.frame(varImp(fm))
write.csv(export, file = "D:\\GEOBIA\\importancia.csv")

In [32]:
### evaluar la exactitud del modelo
pridicted = factor(pred.rf)
real = factor(dataTest$class)
my_data1 <- data.frame(data = pridicted, type = "prediction")
my_data2 <- data.frame(data = real, type = "real")
my_data3 <- rbind(my_data1,my_data2)
identical(levels(my_data3[my_data3$type == "prediction",1]) , levels(my_data3[my_data3$type == "real",1]))

[1] TRUE

In [33]:
### convertir a matriz 
matrix = confusionMatrix(my_data3[my_data3$type == "prediction",1], my_data3[my_data3$type == "real",1],  dnn = c("Prediction", "Reference"))

In [34]:
### visualizar la matriz 
data.matrix(matrix)

,bosque,enero,enmalezado,febrero,humedo,inundado,matorral,seco,urbano
bosque,13,0,0,0,0,0,0,0,1
enero,0,19,0,0,0,0,0,0,0
enmalezado,0,0,17,4,0,0,0,0,0
febrero,0,0,0,17,0,0,0,0,0
humedo,0,0,0,0,16,0,0,0,0
inundado,0,0,0,0,0,17,0,0,0
matorral,0,0,0,0,0,0,21,0,0
seco,0,0,0,0,1,0,0,19,0
urbano,0,0,0,0,0,0,0,1,18


El modelo logró tener una proporción de clasificaciones correctas de un 95.7 % y un índice Kappa de 95.2% casi igual lo que indica que no existió efectos aleatorios que pudieran influir en el resultado, se encontró un segmento definido en los datos de entrenamiento como perteneciente a la clase seco fue encontrado por el modelo como húmedo, de igual manera otro como urbano fue encontrado como seco, mientras que 4 segmentos definidos por el modelo como pertenecientes a siembras en febrero habían sido definidos en el entrenamiento como pertenecientes a lotes enmalezados, lo mismo que uno urbano era definido como bosque.

In [52]:
### visualizar indice Kappa y exactitud global
round(matrix$overall,3)

Accuracy          Kappa  AccuracyLower  AccuracyUpper   AccuracyNull 
         0.957          0.952          0.914          0.983          0.128 
AccuracyPValue  McnemarPValue 
         0.000            NaN

In [41]:
### correr el modelo con la totalidad de los datos 
predicciones_rf = predict(modelo_rf, newdata = data_all, type = "raw")

In [42]:
### convertir el resultado a un data.frame y exportarlo a un csv 
resultado=as.data.frame(predicciones_rf)
write.csv(resultado, file = "D:\\GEOBIA\\clasificacion.csv")